In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

In [3]:
cleaned_data = pd.read_csv("dataset/cleaned_data2.csv")

In [4]:
cleaned_data.head(5)

,reviewerID,reviewText,clean_reviewText,clean_summary,rating_class
0,A1BZ18U2EF4FAI,the nook color is a great tablet. it sometimes...,nook color great tablet sometimes slow still t...,good,good
1,A3W2CJFXG0158G,"1. I have a iPhone3Gs with a case, the cable w...",phone is case win without remove case manage p...,first belkin disappointment,bad
2,A3F9CBHV4OHFBS,I needed this cable to connect my Epson projec...,need connect elson project lawton great price ...,great product especially price,good
3,AZHV16QDXACKC,i bought these over a year ago hoping that a c...,year ago hop company offer lifetime warranty p...,not great,bad
4,A231WM2Z2JL0U3,or those who shun manuals. I am a fairly compu...,shun manual fairly computer-navvy person hande...,not faint-hearted,bad


In [5]:
cleaned_data["rating_class"] = cleaned_data["rating_class"].apply(lambda x: 1 if x=="good" else 0)

In [6]:
cleaned_data.head(5)

,reviewerID,reviewText,clean_reviewText,clean_summary,rating_class
0,A1BZ18U2EF4FAI,the nook color is a great tablet. it sometimes...,nook color great tablet sometimes slow still t...,good,1
1,A3W2CJFXG0158G,"1. I have a iPhone3Gs with a case, the cable w...",phone is case win without remove case manage p...,first belkin disappointment,0
2,A3F9CBHV4OHFBS,I needed this cable to connect my Epson projec...,need connect elson project lawton great price ...,great product especially price,1
3,AZHV16QDXACKC,i bought these over a year ago hoping that a c...,year ago hop company offer lifetime warranty p...,not great,0
4,A231WM2Z2JL0U3,or those who shun manuals. I am a fairly compu...,shun manual fairly computer-navvy person hande...,not faint-hearted,0


In [7]:
Y = cleaned_data["rating_class"]
X = cleaned_data["clean_reviewText"]

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42,shuffle=True, stratify = Y)

In [9]:
tfidf_vect = TfidfVectorizer(ngram_range=(1, 1))
tfidf_vect_train = tfidf_vect.fit_transform(X_train.values.astype("U"))
tfidf_vect_train = tfidf_vect_train.toarray()
tfidf_vect_test = tfidf_vect.transform(X_test.values.astype("U"))
tfidf_vect_test = tfidf_vect_test.toarray()

In [63]:
model = XGBClassifier(n_estimators=200, learning_rate =0.20, max_depth = 30,min_child_weight= 5,gamma = 0.5,colsample_bytree = 0.4   )
eval_s = [(tfidf_vect_train, Y_train), (tfidf_vect_test, Y_test)]
model.fit(tfidf_vect_train, Y_train, eval_metric=["error"], verbose=True, eval_set = eval_s, early_stopping_rounds=20)
Ypred = model.predict(tfidf_vect_test)
accuracy = accuracy_score(Y_test, Ypred)
f1Score = f1_score(Y_test, Ypred, average = "weighted")

[0]	validation_0-error:0.25375	validation_1-error:0.317
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 20 rounds.
[1]	validation_0-error:0.20325	validation_1-error:0.289
[2]	validation_0-error:0.184	validation_1-error:0.296
[3]	validation_0-error:0.1695	validation_1-error:0.288
[4]	validation_0-error:0.14975	validation_1-error:0.272
[5]	validation_0-error:0.142	validation_1-error:0.266
[6]	validation_0-error:0.14375	validation_1-error:0.253
[7]	validation_0-error:0.1325	validation_1-error:0.254
[8]	validation_0-error:0.127	validation_1-error:0.242
[9]	validation_0-error:0.117	validation_1-error:0.255
[10]	validation_0-error:0.112	validation_1-error:0.255
[11]	validation_0-error:0.1045	validation_1-error:0.252
[12]	validation_0-error:0.09775	validation_1-error:0.253
[13]	validation_0-error:0.093	validation_1-error:0.251
[14]	validation_0-error:0.08875	validation_1-error:0.249
[15]	vali

In [64]:
accuracy

0.787

In [65]:
f1Score

0.7869639969154787

In [10]:
model = CatBoostClassifier( learning_rate=0.2, max_depth = 6)
model.fit(tfidf_vect_train, Y_train)
Ypred = model.predict(tfidf_vect_test)
accuracy = accuracy_score(Y_test, Ypred)
f1Score = f1_score(Y_test, Ypred, average = "weighted")

0:	learn: 0.6657927	total: 721ms	remaining: 12m
1:	learn: 0.6436724	total: 1.2s	remaining: 9m 59s
2:	learn: 0.6230373	total: 1.65s	remaining: 9m 6s
3:	learn: 0.6124115	total: 2.09s	remaining: 8m 41s
4:	learn: 0.6026402	total: 2.54s	remaining: 8m 24s
5:	learn: 0.5932102	total: 3.09s	remaining: 8m 31s
6:	learn: 0.5848300	total: 3.53s	remaining: 8m 21s
7:	learn: 0.5790758	total: 4.07s	remaining: 8m 24s
8:	learn: 0.5737003	total: 4.51s	remaining: 8m 16s
9:	learn: 0.5609942	total: 5.03s	remaining: 8m 18s
10:	learn: 0.5560788	total: 5.57s	remaining: 8m 21s
11:	learn: 0.5509558	total: 6.07s	remaining: 8m 19s
12:	learn: 0.5451847	total: 6.5s	remaining: 8m 13s
13:	learn: 0.5390956	total: 7.09s	remaining: 8m 19s
14:	learn: 0.5348610	total: 7.54s	remaining: 8m 15s
15:	learn: 0.5307856	total: 8.02s	remaining: 8m 13s
16:	learn: 0.5261528	total: 8.49s	remaining: 8m 11s
17:	learn: 0.5231433	total: 9.02s	remaining: 8m 12s
18:	learn: 0.5188475	total: 9.61s	remaining: 8m 16s
19:	learn: 0.5130880	total: 

In [11]:
accuracy

0.783

In [12]:
f1Score

0.7829893664789576

In [13]:
model = LogisticRegression()
model.fit(tfidf_vect_train, Y_train)
Ypred = model.predict(tfidf_vect_test)
accuracy = accuracy_score(Y_test, Ypred)
f1Score = f1_score(Y_test, Ypred, average = "weighted")

In [14]:
accuracy

0.802

In [15]:
f1Score

0.8019928717433829